In [1]:
import os
import csv
import sys
import random
import scipy
import numpy as np
import xgboost as xgb
# read in data


workspace_path = '/home/irashadow/python_workspace/Kaggle_Comp/Santander/'
month_dict_inv = dict()
month_dict_inv[0] = '2015-01'
month_dict_inv[1] = '2015-02'
month_dict_inv[2] = '2015-03'
month_dict_inv[3] = '2015-04'
month_dict_inv[4] = '2015-05'
month_dict_inv[5] = '2015-06'
month_dict_inv[6] = '2015-07'
month_dict_inv[7] = '2015-08'
month_dict_inv[8] = '2015-09'
month_dict_inv[9] = '2015-10'
month_dict_inv[10] = '2015-11'
month_dict_inv[11] = '2015-12'
month_dict_inv[12] = '2016-01'
month_dict_inv[13] = '2016-02'
month_dict_inv[14] = '2016-03'
month_dict_inv[15] = '2016-04'
month_dict_inv[16] = '2016-05'
month_dict_inv[17] = '2016-06'

/home/irashadow/env_TensorFlow/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [11]:
cust_2_labels_dict = dict()

pos_case_dict = dict()
neg_case_dict = dict()


train_write = open(workspace_path + '/train/cust_product_mc.train', 'w')

check_line_count = 0    
for date_index_major in [5,6,16]:
#for date_index_major in [5,13,16]:
    
    feat_file = open(workspace_path + '/L4_feat_added/4_7_cust_date_nominal_prod_L4_'+str(month_dict_inv[date_index_major])+'_Model.feat', 'r')
    
    for line in feat_file:

        line = line.strip()
        check_line_count += 1

        if len(line.split('|')) < 6:

           print line 

        cust_id = line.split('|')[0]
        date_note = line.split('|')[1]
        label_tokens = line.split('|')[2]
        
        get_label = np.zeros(30)

        for label_item in label_tokens.split(','):

            if len(label_item) > 0:

                prod_id = int(label_item)
                get_label[prod_id] = 1


        if True:

           for j in range(1,25):
            
                
                  if get_label[j] == 1:
 
                     prt_line = str(j-1) + ' ' 
                     for fi in range(3,8):
                         if len(line.split('|')[fi]) > 0:
                            prt_line += line.split('|')[fi] + ' ' 

                     train_write.write(prt_line[:-1]+'\n') 
                    
       
        
    feat_file.close()           
 
train_write.close()               
print check_line_count    

105751


In [12]:
train_data_path = './train/cust_product_mc.train'

dtrain = xgb.DMatrix(train_data_path)
dvalid = xgb.DMatrix(train_data_path)

# specify parameters via map
param ={"objective" : "multi:softprob",    # multiclass classification 
              "num_class" : 24,    # number of classes 
              "eval_metric" : "mlogloss",    # evaluation metric 
              "nthread" : 6,   # number of threads to be used 
              "max_depth" : 12,    # maximum depth of tree 
              "eta" : 0.05,    # step size shrinkage 
              "subsample" : 0.8,    # part of data instances to grow tree 
              "colsample_bytree" : 0.8  # subsample ratio of columns when constructing each tree 
}
num_round = 50
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dvalid)

print preds.shape

(130892, 24)


In [13]:
preds_sort = np.argsort(preds, axis=1)
print preds_sort.shape
preds_top7 = np.fliplr(preds_sort)[:,:7]
print preds_top7

#----------------------------------------------------
dtrain_label = dtrain.get_label().astype(int)
print dtrain_label[:10].astype(int)
#preds = np.fliplr(preds)[:,:7]

(130892, 24)
[[ 2 21 22 ..., 11  8  7]
 [ 2 11 18 ..., 13 22 21]
 [22 14 21 ...,  7 11 19]
 ..., 
 [ 2  6 23 ..., 21  9 11]
 [ 2  4  9 ..., 23 12 22]
 [ 2  6 23 ..., 21 22  4]]
[ 2  2 22 22 23  2 17 21 22 21]


In [14]:
correct_count = 0

for i in range(0,len(dtrain_label)):
    
    if preds_top7[i,0] == dtrain_label[i]:

       correct_count += 1 

print 'accuracy:%.3f'%(correct_count/float(len(dtrain_label)))

accuracy:0.673


In [15]:
test_data_path = './test/5_4_cust_date_prod_L4.test'
dtest = xgb.DMatrix(test_data_path)
preds = bst.predict(dtest)

In [16]:
preds_sort = np.argsort(preds, axis=1)
print preds_sort.shape
preds_top7 = np.fliplr(preds_sort)[:,:15]
print preds_top7

label_submit_dict = dict()
for i in range(0,preds_sort.shape[0]):
#for i in range(0,100):
    
    top_product = preds_top7[i,0]
    
    if top_product not in label_submit_dict:
       label_submit_dict[top_product]  = 1
    else:
       label_submit_dict[top_product]  += 1
    

(929615, 24)
[[23 11 12 ...,  6 15 14]
 [23 17 22 ...,  7  8 15]
 [23 21 22 ...,  8  7  5]
 ..., 
 [23 18 11 ...,  6 15  2]
 [23 17 18 ...,  6  2  5]
 [ 2 23 18 ...,  8  7  6]]


In [17]:
print 'submission distribution'

for item in label_submit_dict:
    
    print 'label #' + str(item+1) + ' :%.2f'%(label_submit_dict[item]/float(preds_sort.shape[0])*100.0) 
    #print 'label #' + str(item)


submission distribution
label #3 :33.01
label #4 :0.00
label #5 :0.36
label #6 :0.11
label #7 :0.00
label #8 :0.07
label #9 :0.03
label #10 :0.01
label #11 :0.00
label #12 :0.15
label #13 :1.10
label #14 :0.02
label #16 :0.00
label #18 :2.16
label #19 :5.26
label #20 :0.04
label #21 :0.00
label #22 :1.37
label #23 :4.30
label #24 :52.01


In [18]:
label_dict = dict()
label_dict[1] = 'ind_ahor_fin_ult1'
label_dict[2] = 'ind_aval_fin_ult1'
label_dict[3] = 'ind_cco_fin_ult1'
label_dict[4] = 'ind_cder_fin_ult1'
label_dict[5] = 'ind_cno_fin_ult1'
label_dict[6] = 'ind_ctju_fin_ult1'
label_dict[7] = 'ind_ctma_fin_ult1'
label_dict[8] = 'ind_ctop_fin_ult1'
label_dict[9] = 'ind_ctpp_fin_ult1'
label_dict[10] = 'ind_deco_fin_ult1'
label_dict[11] = 'ind_deme_fin_ult1'
label_dict[12] = 'ind_dela_fin_ult1'
label_dict[13] = 'ind_ecue_fin_ult1'
label_dict[14] = 'ind_fond_fin_ult1'
label_dict[15] = 'ind_hip_fin_ult1'
label_dict[16] = 'ind_plan_fin_ult1'
label_dict[17] = 'ind_pres_fin_ult1'
label_dict[18] = 'ind_reca_fin_ult1'
label_dict[19] = 'ind_tjcr_fin_ult1'
label_dict[20] = 'ind_valo_fin_ult1'
label_dict[21] = 'ind_viv_fin_ult1'
label_dict[22] = 'ind_nomina_ult1'
label_dict[23] = 'ind_nom_pens_ult1'
label_dict[24] = 'ind_recibo_ult1'

In [19]:
Eout_write = open('./output/TheLastFourRecord_pred_Dec03_xgb.eout', 'w')  

feat_data_path = workspace_path + '/input/test_ver2.csv'

test_read = open(feat_data_path, 'r')
headline = test_read.readline().strip()

line_count = 0
for line in test_read:
    
    line = line.strip().split(',')
    cust_id = int(line[1].replace(' ',''))
    
    prt_line_temp = ''
    
    for i in range(0,13):
        
        product_index = preds_top7[line_count,i]
        
        prt_line_temp += label_dict[product_index+1] + ' '
        
    prt_line_temp = prt_line_temp[:-1]    
    
    
    prt_line = str(cust_id) + '|' + prt_line_temp + '\n'
    
    
    line_count += 1
    
    Eout_write.write(prt_line)
   
print line_count 
Eout_write.close()
    

929615


In [20]:
dict_read = open(workspace_path + '/dict/cust_last_prods.txt', 'r')

cust_end_products_dict = dict()

line_count = 0
for line in dict_read:  
    line = line.strip()
    
    line_count += 1
    
    cust_id = int(line.split('|')[0])
    last_prod_date = line.split('|')[1]
    label_tokens_get = line.split('|')[2]

    
    if cust_id not in cust_end_products_dict:
       cust_end_products_dict[cust_id] = [] 
    
    if len(label_tokens_get) > 0 and last_prod_date == '2016-05': 
    
        for feat_item in label_tokens_get.split(','):
            
            prod_id = int(feat_item)
            cust_end_products_dict[cust_id].append(label_dict[prod_id])
        
print line_count 
dict_read.close()

956645


In [21]:
Eout_dict_read = open('./output/TheLastFourRecord_pred_Dec03_xgb.eout', 'r')  
id_2_prod_dict = dict()

for line in Eout_dict_read:
    line = line.strip()
    
    if len(line.split('|')) != 2:
       print 'error_line:'+line
    else:
       cust_id  = int(line.split('|')[0])
       products = line.split('|')[1]
        
       id_2_prod_dict[cust_id] = products 

    
sample_submission_read = open('./output/sample_submission.csv', 'r')

head_line = 'ncodpers,added_products' + '\n'
submission_write = open('./output/submission_Dec03-2.csv', 'w')  
submission_write.write(head_line)

check_line_count = 0

duplicated_items_count = 0
#for i in range(0,10000):    
    #line = sample_submission_read.readline().strip()
    
for line in sample_submission_read:
    line = line.strip()    
    
    if 'ncodpers' not in line:
    
        cust_id  = int(line.split(',')[0])

        prt_line = str(cust_id) + ','
        
        pruned_line = False
        
        added_items_count = 0

        if cust_id in id_2_prod_dict:
            
           
           #print cust_end_products_dict[cust_id]
           #print 'predict:'+ str(id_2_prod_dict[cust_id])  
           
           for prod_name_item in id_2_prod_dict[cust_id].split(' '):
            
               
               if cust_id in cust_end_products_dict:
                   if prod_name_item not in cust_end_products_dict[cust_id]:
                        
                         if added_items_count < 7:
                            
                            prt_line += prod_name_item + ' '
                        
                            added_items_count += 1

                            pruned_line = True

                   else: 

                     #print '->' + prod_name_item    

                        duplicated_items_count += 1

        check_line_count += 1
        
        if pruned_line:
           submission_write.write(prt_line[:-1]+'\n')
        else:
           submission_write.write(prt_line+'\n')  

print check_line_count 
print duplicated_items_count
    
submission_write.close() 
sample_submission_read.close() 
Eout_dict_read.close() 

929615
570612
